<a href="https://colab.research.google.com/github/zwfswt/GEE/blob/main/Planet_thumbs2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install planet

     |████████████████████████████████| 55 kB 1.1 MB/s 
  Created wheel for requests-futures: filename=requests_futures-0.9.7-py3-none-any.whl size=5073 sha256=2fcc1c6e8f2e8e36faf97d40bca2ddc60076a0e84d5f1f5d4c9c200be1c77b29
  Stored in directory: /root/.cache/pip/wheels/09/59/e0/d5725c111058931dcb914bd3fa226f7a52d419de101dda4264
Successfully built requests-futures


In [2]:
from planet import api
import requests

In [3]:
client = api.ClientV1(api_key="PLAK9948855dd01246448636bc5229a3ec73")

In [4]:
import json
def p(data):
    print(json.dumps(data, indent=2))

In [5]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files,drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [6]:
import os
path = "/content/drive/My Drive/Planet"
os.chdir(path)

In [15]:
!ls

example1.geojson  example2.geojson  example.geojson


In [8]:
aoi = {
  "type": "Polygon",
  "coordinates": [
    [
      [120.699216,32.169942],
      [120.699216,33.854232],
      [122.70791,33.854232],
      [122.70791,32.169942],
      [120.699216,32.169942]
    ]
  ]
}

In [16]:
from planet.api import filters

In [17]:
from datetime import datetime
geometry_filter=filters.geom_filter(aoi)
start_date = datetime(year=2022, month=5, day=3)
end_date = datetime(year=2022, month=5, day=3)
date_filter = filters.date_range('acquired', gte=start_date,lte=end_date)
cloud_filter = filters.range_filter('cloud_cover', lte=0.5)


In [18]:
and_filter = filters.and_filter(geometry_filter,date_filter, cloud_filter)

In [19]:
# item_types = ["REOrthoTile", "PSOrthoTile"]
item_types = ["PSScene"]
request = filters.build_search_request(and_filter, item_types)

In [20]:
results = client.quick_search(request)

In [14]:
from IPython.display import display, Image
filenames = []
NUM_THUMBNAILS = 100
IMAGE_WIDTH = 2048
for item in results.items_iter(NUM_THUMBNAILS):
    thumbnail_url = item['_links']['thumbnail']
    # assets = client.get_assets(item).get()
    
    # print(thumbnail_url)
    # response = requests.get(
    #     thumbnail_url,params=dict(width=IMAGE_WIDTH)
    # )
    api_key="PLAK9948855dd01246448636bc5229a3ec73"
    response = requests.get(thumbnail_url,auth=(api_key, ''),params=dict(width=IMAGE_WIDTH))
    # print(response.content)

    filename = '_'.join(
        [item['id'], item['properties']['item_type'],]
    )
    filename += '.png'
    with open(filename, 'wb') as f:
        f.write(response.content)
    filenames.append(filename)
    print(filename)
    display(Image(filename=filename,width = 300, height = 300))